In [ ]:
### 5.1 读写文本数据

我们需要对文本数据进行读写操作， 但这个过程有可能针对不同的文本编码进行。   
可以使用open()函数配合rt模式来读取文本文件的内容。
```python
#Read the entire file as a single string
with open('somefile.txt', 'rt') as f:
    data = f.read()
   
# Iterate over the lines of the file
with open('somefile.txt', 'rt') as f:
    for line in f:
        # process line
        pass
```
    

### 5.2 将输出重定向到文件中

如下：
```python
with open('somefile.txt', 'rt') as f:
    print('Hello World!', file = f)
```

### 5.3 以不同的分隔符或行结尾符完成打印

In [1]:
print('ACME', 50, 91.5)
print('ACME', 50, 91.5, sep = ',', end = '!!\n')

ACME 50 91.5
ACME,50,91.5!!


### 5.4 读写二进制数据
```python
with open('somfile.bin', 'rb') as f:
    data = f.read()
```

特别要注意的是， 在做索引和迭代操作时，字节串会返回代表该字节的整数值而不是字节串。如下：

In [2]:
t = 'Hello World'
print(t[0])
for c in t:
    print(c, end=' ')

H
H e l l o   W o r l d 

In [3]:
b = b'Hello World'
print(b[0])
for c in b:
    print(c, end = ' ')

72
72 101 108 108 111 32 87 111 114 108 100 

如果要在二进制文件中读取或写入文本内容，请确保要进行编码或解码操作。
```python
with open('somefile.bin', 'rb') as f:
    data = f.read(16)
    text = data.decode('utf-8')
    
with open('somefile.bin', 'wb') as f:
    text = 'Hello world'
    f.write(text.encode('utf-8')
    
```

### 5.5 对已不存在的文件执行写入操作

```python
import os 
if not os.path.exists('somefile'):
    with open('somefile', 'wt') as f:
    f.write('Hello\n')
```
等同如下代码：
```python
with opne('somefile', 'xt') as f:
    f.write('Hello\n')
```

### 5.6 在字符串上执行I/O操作

使用io.StringIO()和io.BytesIO()类来创建类似于文件的对象，这些对象可操作字符串数据。

In [3]:
import io
s = io.StringIO()
s.write('Hello World\n')
print('This is a test', file = s)
s.getvalue()

'Hello World\nThis is a test\n'

### 5.7 读写压缩的数据文件

把压缩的文件以文本形式读取

```python
# gzip compression
import gzip
with gzip.open('somefile.gz', 'rt') as f:
    text = f.read()
    
# bz2 compression
import bz2
with bz2.open('somefile2.bz2', 'rt') as f:
    text = f.read()
```
要写入压缩数据，可以这样处理：
```python
# gzip compression
import gzip
with gzip.open('somefile.gz', 'wt') as f:
    f.write(text)
    
# bz2 compression
import bz2
with bz2.open('somefile2.bz2', 'wt') as f:
    f.write(text)
```


### 5.8 对固定大小的记录进行迭代

与其按行来迭代文件，我们想对一系列固定大小的记录或数据块进行迭代。

```python
from functools import partial
RECORD_SIZE = 32
with open('somefile.data', 'rb') as f:
    records = iter(partial(f.read, RECORD_SIZE), b'')
    for r in records:
        pass
```

### 5.9 将二进制数据读取到可变缓冲区中

我们想将二进制数据直接读取到一个可变缓冲区中，中间不经过任何拷贝环节。  
也许我们想原地修改数据再将它写会到文件中去。

In [4]:
import os.path
def read_into_buffer(filename):
    buf = bytearray(os.path.getsize(filename))
    with open(filename, 'rb') as f:
        f.readinto(buf)
    return buf

In [5]:
with open('sample.bin', 'wb') as f:
    f.write(b'Hello World')

In [6]:
buf = read_into_buffer('sample.bin')

In [7]:
buf

bytearray(b'Hello World')

In [8]:
buf[0:5] = b'Hallo'
print(buf)

bytearray(b'Hallo World')


In [9]:
with open('newsample.bin', 'wb') as f:
    f.write(buf)

文件对象的readinto（）方法可用来将数据填充到任何预分配好的数组中，这包括array模块或者numpy这样的库所创建的数组。与普通的read（）方法不同的是，reainto()是为已经存在的缓冲区填充内容，而不是分配新的对象然后再将它们返回。因此，可以用readinto（）来避免产生额外的内存分配动作。

### 5.10 对二进制文件做内存映射

In [12]:
import os
import mmap
def memory_map(filename, access=mmap.ACCESS_WRITE):
    size = os.path.getsize(filename)
    fd = os.open(filename, os.O_RDWR)
    return mmap.mmap(fd, size, access=access)

size = 1000000
with open('data', 'wb') as f:
    f.seek(size-1)
    f.write(b'\x00')

In [20]:
m = memory_map('data')

In [21]:
len(m)

1000000

In [22]:
m[0:10]

b'Hello Worl'

In [23]:
m[0]

72

In [24]:
m[0:11] = b'Hello World'

In [25]:
m.close()

In [26]:
with open('data', 'rb') as f:
    print(f.read(11))

b'Hello World'


默认情况下，memory_map()函数打开的文件既可以读也可写。对数据的任何修改都会拷贝回原始文件中。如果需要只读访问，可以为access参数提供mmap.ACCESS_READ值。
```python
m = memory_map(filename, mmap.ACCESS_READ)
```
如果只想在本地修改数据，并不想将这些修改写回到原始文件中，可以使用mmap.ACCESS_COPY参数：
```python
m = memory_map(filename, mmap.ACCESS_COPY)
```

### 5.11 处理路径名

In [27]:
import os
print(os.path.abspath(os.curdir))

E:\pywork\Python\CookBook\5. 文件和IO


In [28]:
path = 'E:\\pywork\\Python\\CookBook\\5. 文件和IO/sample.bin'

In [29]:
os.path.basename(path)

'sample.bin'

In [30]:
os.path.dirname(path)

'E:\\pywork\\Python\\CookBook\\5. 文件和IO'

In [31]:
os.path.join('tmp', 'data', os.path.basename(path))

'tmp\\data\\sample.bin'

In [32]:
path = '~/data/data.csv'
os.path.expanduser(path)

'C:\\Users\\wangchx/data/data.csv'

In [33]:
os.path.splitext(path)

('~/data/data', '.csv')

### 5.12 检测文件是否存在

In [34]:
import os
os.path.exists('/etc/passwd')

False

In [35]:
import os
path = 'E:\\pywork\\Python\\CookBook\\5. 文件和IO/sample.bin'
os.path.exists(path)

True

In [36]:
os.path.isfile(path)

True

In [37]:
os.path.isdir(path)

False

In [38]:
os.path.islink(path)

False

In [39]:
os.path.realpath(path)

'E:\\pywork\\Python\\CookBook\\5. 文件和IO\\sample.bin'

In [40]:
os.path.getsize(path)

11

In [41]:
os.path.getmtime(path)

1569208006.2854955

### 5.13  获取目录内容的列表

In [44]:
import os
names = os.listdir(os.path.expanduser('~'))

In [45]:
print(names)

['.anaconda', '.android', '.astropy', '.conda', '.condarc', '.eclipse', '.idlerc', '.ipython', '.jupyter', '.matplotlib', '.p2', '.PyCharmCE2018.2', '.recently-used.xbel', '.spyder-py3', '.webclipse', '.webclipse.properties', '123.py', '3D Objects', 'AppData', 'Application Data', 'arraysortedbag.py', 'Contacts', 'Cookies', 'Desktop', 'Documents', 'Downloads', 'Favorites', 'IntelGraphicsProfiles', 'Links', 'Local Settings', 'MicrosoftEdgeBackups', 'Music', 'My Documents', 'NetHood', 'new.txt', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{a2c2c18e-6978-11e8-b989-8cec4b7c4b1d}.TM.blf', 'NTUSER.DAT{a2c2c18e-6978-11e8-b989-8cec4b7c4b1d}.TMContainer00000000000000000001.regtrans-ms', 'NTUSER.DAT{a2c2c18e-6978-11e8-b989-8cec4b7c4b1d}.TMContainer00000000000000000002.regtrans-ms', 'ntuser.ini', 'OneDrive', 'Pictures', 'PrintHood', 'Recent', 'Saved Games', 'Searches', 'SendTo', 'Templates', 'untitled0.py', 'Videos', 'xmltest.xml', '「开始」菜单']


### 5.14  绕过文件名编码

In [46]:
import sys
print(sys.getfilesystemencoding())

utf-8


In [47]:
# write a file using a unicode filename
with open('jalape\xf1o.txt', 'w') as f:
    f.write('Spicy!')

In [48]:
# Directory listing(decoded)
import os
os.listdir('.')

['.ipynb_checkpoints',
 'data',
 'jalapeño.txt',
 'newsample.bin',
 'sample.bin',
 'Untitled.ipynb']

In [49]:
# Directoy listing(raw)
os.listdir(b'.')

[b'.ipynb_checkpoints',
 b'data',
 b'jalape\xc3\xb1o.txt',
 b'newsample.bin',
 b'sample.bin',
 b'Untitled.ipynb']

In [50]:
# open file with raw filename
with open(b'jalape\xc3\xb1o.txt') as f:
    print(f.read())

Spicy!


In [3]:
a = [1,2, 3, 5]
b = [2,3]
sorted((a,b), key=lambda x : len(x), reverse=True)

[[1, 2, 3, 5], [2, 3]]

### 5.15 打印无法解码的文件名

```python
def bad_filename(filename):
    return repr(filename)[1:-1]
try:
    print(filename)
except UnicodeEncodeError:
    print(bad_filename(filename)
```

```python
def bad_filename(filename):
    temp = filename.encode(sys.getfilesystemencoding(), errors='surrogateescape')
    return temp.decode('latin-1')
```

### 5.16  为已经打开的文件添加或修改编码方式

如果为一个以二进制模式打开的文件对象添加Unicode编码/解码，可以用io.TextIOWrapper()对象将其包装。如下：
```python
import urllib.request
import io
u = urllib.request.urlopen('http://www.python.org')
f = io.TextIOWrapper(u, encoding='utf-8')
text = f.read()
```

如果想修改一个已经以文本模式打开的文件的编码方式，可以在用新的编码替换之前的编码前，用detach()方法将已有的文本编码层移除。  
如下是修改sys.stdout编码的例子。

In [1]:
import sys
sys.stdout.encoding

'UTF-8'

```python
import io
sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='latin-1')
```

IO系统是一系列层次来构建的。
* io.TextIOWrapper是一个文本处理层，它负责编码和解码Unicode。
* io.BufferedWriter是一个缓冲IO层，负责处理二进制数据。
* io.FileIO是一个原始文件，代表着操作系统底层的文件描述符。 

添加或修改文本的编码涉及添加或修改最上层的io.TextIOWrapper层。

In [5]:
f = open('sample.bin', 'w')

In [6]:
f

<_io.TextIOWrapper name='sample.bin' mode='w' encoding='cp936'>

In [7]:
f.buffer

<_io.BufferedWriter name='sample.bin'>

In [8]:
f.buffer.raw

<_io.FileIO name='sample.bin' mode='wb' closefd=True>

如果直接访问buffer层并修改编码是不安全的，也不起作用，因为f之前的值已经被销毁。  
detach()方法将最上层的Text层同文件分离开来，并将下一个层次buffer返回。

In [10]:
b = f.detach()

In [11]:
b

<_io.BufferedWriter name='sample.bin'>

In [13]:
f = io.TextIOWrapper(b, encoding = 'latin-1')

In [14]:
f

<_io.TextIOWrapper name='sample.bin' encoding='latin-1'>

In [19]:
f.close()

### 5.17 将字节数据写入文本文件

只需要简单的将字节数据写入到文件底层的buffer中就可以了。

In [25]:
f = open('sample.bin', 'wt')

In [26]:
f.write(b'Hello\n')

TypeError: write() argument must be str, not bytes

In [27]:
f.buffer.write(b'Hello\n')

6

In [28]:
f.close()

### 5.18 将已有的文件描述符包装为文件对象

文件描述符与一般打开文件相比还是有区别的。文件描述符只是一个由操作系统分配的整数句柄，用来指代某种系统IO通道。如果刚好有这样一个文件描述符，就可以通过open（）函数用python文件对象对齐进行包装。

In [30]:
import os
fd = os.open('somefile.txt', os.O_WRONLY | os.O_CREAT)

# Turn into a proper file
f = open(fd, 'wt')
f.write('hello world\n')
f.close()

当高层的文件对象被关闭或销毁时，底层的文件描述符也会被关闭。  
如果不想这样，只需给open()函数提供一个可选的closefd=False参数即可。

在UNIX系统上，这种包装文件描述符的技术可以用来方便地对不同方式打开地IO通道（管道、socket等）提供一个类似于文件地接口。

In [ ]:
from socket import socket,AF_INET, SOCK_STREAM

def echo_client(client_sock, addr):
    print('Got connection from', addr)